Required packets

In [1]:
from pandas import read_csv, DataFrame
import pywt
import matplotlib.pyplot as plt
import numpy as np

## Wavelet Transform

Apply CWT to match eventually the right seasonality, mother wavelet used is Morlet.

Load the dataset

In [ ]:
series = read_csv('series_normalized.csv', header=0, index_col=0, parse_dates=True).squeeze("columns")

# Create a date range with the same start and end dates and frequency as the original series
time_index = series.index

# prepare data
X = series.values.astype('float32')

Plot

In [ ]:
plt.plot(time_index, X)
plt.title('Dataset')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.show()

Select the right frequencies and normalize based on sampling frequency

In [ ]:
# Calculate the time difference between consecutive timestamps
time_diff = series.index.to_series().diff().min()

# Set dt based on the time difference
dt = time_diff.total_seconds()

# Convert dt to Hertz
fs = 1 / dt

# Define the frequencies for daily to yearly
frequencies = np.array([0.0000115740741, 1.6534391534391535e-06, 4.133597883597884e-07, 3.21502058e-8])

# Select frequencies lower than or equal to the sampling frequency
selected_frequencies = frequencies[frequencies < fs] / fs

Possible Wavelet

In [ ]:
wavlist = pywt.wavelist(kind='continuous')
print(wavlist)

Wavelet Transform based on scales obtained from selected frequencies

In [ ]:
scales = pywt.frequency2scale('morl', selected_frequencies)
coef, freqs=pywt.cwt(X, scales, 'morl')

Create the new dataframe with the coef values for each scale

In [ ]:
if len(freqs) == 4:
  column_names = ['Daily', 'Weekly', 'Monthly', 'Yearly']
elif len(freqs) == 3:
  column_names = ['Weekly', 'Monthly', 'Yearly']
elif len(freqs) == 2:
  column_names = ['Monthly', 'Yearly']
else:
  column_names = ['Yearly']

# Create a DataFrame with the time index and coefficients
df_coef = DataFrame(data= coef.T, index=time_index, columns=[f'{name}' for name in column_names])
# Set the index name
df_coef.index.name = 'Date'
#df_coef_abs = DataFrame(data= np.abs(coef.T), index=time_index, columns=[f'Freq_{freq}' for freq in freqs])

# Save the DataFrame to a CSV file
df_coef.to_csv('coefficents.csv')
#df_coef_abs.to_csv('abs_coefficents.csv')

Plot the scalogram

In [ ]:
plt.imshow(coef, aspect='auto', extent=[time_index[0], time_index[-1], freqs[0], freqs[-1]], cmap='jet', interpolation='bilinear')
plt.title('Scalogram')
plt.xlabel('Time')
plt.ylabel('Frequency (Hz)')
plt.colorbar(label='Amplitude')
plt.show()

Plot the Wavelet Transform for each scale

In [ ]:
for i in range(len(coef)):
  plt.plot(time_index, coef[i])
  plt.title(f'morl- freq {freqs[i]}')
  plt.xlabel('Time')
  plt.ylabel('Amplitude')
  plt.show()